In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
import itertools
import copy
from tqdm import tqdm
from weather_data_pipeline import WeatherPipeline, DatasetConfig
from weather_feature_pipeline import FeaturePipeline

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
def tensorfy(x):
    if torch.is_tensor(x):
        return x.clone().detach().float()
    return torch.from_numpy(x).float()

In [4]:
class WeatherModel(nn.Module):
    def __init__(self, input_size, hidden_sizes, output_size_temp, output_size_wind):
        super(WeatherModel, self).__init__()
        
        layer_sizes = [input_size] + list(hidden_sizes)
        layers = []
        
        # Create layers with proper initialization
        for in_size, out_size in itertools.pairwise(layer_sizes):
            linear = nn.Linear(in_size, out_size)
            # Use He initialization for ReLU
            # nn.init.kaiming_uniform_(linear.weight, nonlinearity='relu')
            # nn.init.zeros_(linear.bias)
            
            layers.extend([
                linear,
                # nn.BatchNorm1d(out_size),
                nn.ReLU()
            ])
        
        self.fc1 = nn.Sequential(*layers)
        
        # Temperature output (regression)
        self.fc2_temp = nn.Linear(layer_sizes[-1], output_size_temp)
        # nn.init.xavier_uniform_(self.fc2_temp.weight)
        # nn.init.zeros_(self.fc2_temp.bias)
        
        # Wind output (binary classification)
        self.fc2_wind = nn.Linear(layer_sizes[-1], output_size_wind)
        # nn.init.xavier_uniform_(self.fc2_wind.weight)
        # nn.init.zeros_(self.fc2_wind.bias)
        
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = tensorfy(x)
        out = self.fc1(x)
        temp = self.fc2_temp(out)
        wind = self.sigmoid(self.fc2_wind(out))
        return temp.squeeze(), wind.squeeze()

In [5]:
class EarlyStopping:
    """Early stopping to stop the training when the loss does not improve.
    This is to prevent overfitting and save the best model.
    
    Args:
        patience (int): How long to wait after last time validation loss improved.
        verbose (bool): If True, prints a message for each validation loss improvement.
        delta (float): Minimum change in monitored quantity to qualify as an improvement.
        path (str): Path for the checkpoint to be saved to.
        trace_func (callable): trace print function.
    """
    def __init__(
        self, 
        patience: int = 7,
        verbose: bool = False,
        delta: float = 0,
        path: str = 'checkpoint.pt',
        trace_func = print
    ):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
        self.best_model = None
    
    def __call__(self, val_loss: float, model: torch.nn.Module) -> bool:
        score = -val_loss  # We want to maximize the negative loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0
        
        return self.early_stop
    
    def save_checkpoint(self, val_loss: float, model: torch.nn.Module):
        """Save model when validation loss decrease."""
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model ...')
        
        # Save a deep copy of the model state
        self.best_model = copy.deepcopy(model)
        if self.path:
            torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss
    
    def load_best_model(self) -> torch.nn.Module:
        """Return the best model found during training."""
        if self.best_model is None:
            raise RuntimeError("No model has been saved yet.")
        return self.best_model

In [6]:
def get_optimizer_config(optimizer_name: str = 'sgd'):
    """Get optimizer configuration based on optimizer name.
    
    Args:
        optimizer_name: Either 'sgd' or 'adam'
        
    Returns:
        tuple: (optimizer, scheduler, optimizer_params)
    """
    base_config = {
        'weight_decay': 1e-4,  # L2 regularization for both optimizers
    }
    
    if optimizer_name.lower() == 'sgd':
        optimizer_class = optim.SGD
        optimizer_params = {
            **base_config,
            'lr': 0.001,
            'momentum': 0.9,
            'nesterov': True,
        }
    elif optimizer_name.lower() == 'adam':
        optimizer_class = optim.Adam
        optimizer_params = {
            **base_config,
            'lr': 0.001,
            'betas': (0.9, 0.999),
            'eps': 1e-8,
        }
    else:
        raise ValueError(f"Unsupported optimizer: {optimizer_name}")
    
    def create_optimizer(params):
        optimizer = optimizer_class(params, **optimizer_params)
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode='min',
            factor=0.5,
            patience=10
        )
        return optimizer, scheduler
    
    return create_optimizer

In [ ]:
def train_model(model, feature_pipeline, train_data, val_data,
               criterion_temp, criterion_wind, optimizer, scheduler=None,
               num_epochs=10_000, patience=20, scale_features=False):
    
    X_train = train_data["X"]
    X_val = val_data["X"]
    if scale_features:
        X_train = feature_pipeline.scale_features(X_train, feature_names)
        X_val = feature_pipeline.scale_features(X_val, feature_names)
    
    X_train = tensorfy(X_train).to(device)
    X_val = tensorfy(X_val).to(device)
    y_temp_train = tensorfy(train_data["y_temp"]).to(device)
    y_wind_train = tensorfy(train_data["y_wind"]).to(device)
    y_temp_val = tensorfy(val_data["y_temp"]).to(device)
    y_wind_val = tensorfy(val_data["y_wind"]).to(device)
    
    history = {'train_loss': [], 'val_loss': [], 'learning_rates': []}
    early_stopping = EarlyStopping(patience=patience, verbose=False, delta=1e-4)
    
    for epoch in (pbar := tqdm(range(num_epochs), desc='Training')):
        current_lr = optimizer.param_groups[0]['lr']
        history['learning_rates'].append(current_lr)
        
        model.train()
        optimizer.zero_grad()
        
        outputs_temp, outputs_wind = model(X_train)
        
        if scale_features:
            unscaled_temp_pred = feature_pipeline.inverse_scale_features(outputs_temp.detach().cpu().numpy(), feature_type="temperature")
            unscaled_temp_pred = tensorfy(unscaled_temp_pred).to(outputs_temp.device)
            loss_temp = criterion_temp(unscaled_temp_pred, y_temp_train)
        else:
            loss_temp = criterion_temp(outputs_temp, y_temp_train)
            
        loss_wind = criterion_wind(outputs_wind, y_wind_train)
        loss = loss_temp + loss_wind
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()
        
        model.eval()
        with torch.no_grad():
            val_outputs_temp, val_outputs_wind = model(X_val)
            
            if scale_features:
                unscaled_val_temp = feature_pipeline.inverse_scale_features(val_outputs_temp.cpu().numpy(), feature_type="temperature")
                unscaled_val_temp = tensorfy(unscaled_val_temp).to(val_outputs_temp.device)
                val_loss_temp = criterion_temp(unscaled_val_temp, y_temp_val)
            else:
                val_loss_temp = criterion_temp(val_outputs_temp, y_temp_val)
                
            val_loss_wind = criterion_wind(val_outputs_wind, y_wind_val)
            val_loss = val_loss_temp + val_loss_wind
        
        if scheduler is not None:
            if isinstance(scheduler, optim.lr_scheduler.ReduceLROnPlateau):
                scheduler.step(val_loss)
            else:
                scheduler.step()
        
        history['train_loss'].append(loss.item())
        history['val_loss'].append(val_loss.item())
        
        if (epoch + 1) % 10 == 0:
            pbar.set_description(
                f'Epoch [{epoch+1}/{num_epochs}], '
                f'Loss: {loss.item():.4f}, '
                f'Val Loss: {val_loss.item():.4f}, '
                f'LR: {current_lr:.6f}'
            )
        
        if early_stopping(val_loss.item(), model):
            print(f'Early stopping triggered at epoch {epoch+1}')
            break
    
    model = early_stopping.load_best_model()
    return model, history

In [ ]:
def create_and_train_model(train_dataset: dict[str, dict[str, np.ndarray]], 
                          val_dataset: dict[str, dict[str, np.ndarray]],
                          optimizer_name: str = 'sgd',
                          feature_pipeline = None,
                          scale_features: bool = False,
                          n_epochs: int = 10_000):
    """Create and train weather prediction model."""
    # Create model
    model = WeatherModel(
        input_size=train_dataset['X'].shape[1],
        hidden_sizes=(32,),
        output_size_temp=1,
        output_size_wind=1
    ).to(device)

    # Loss functions
    criterion_temp = nn.MSELoss()
    criterion_wind = nn.BCELoss()

    # Get optimizer configuration
    optimizer_creator = get_optimizer_config(optimizer_name)
    optimizer, scheduler = optimizer_creator(model.parameters())

    # Train model
    model, history = train_model(
        model=model,
        feature_pipeline=feature_pipeline,
        train_data=train_dataset,
        val_data=val_dataset,
        criterion_temp=criterion_temp,
        criterion_wind=criterion_wind,
        optimizer=optimizer,
        scheduler=scheduler,
        num_epochs=n_epochs,
        patience=20,
        scale_features=scale_features
    )

    return model, history

In [ ]:
def predict(model, feature_pipeline, input_data: np.ndarray, feature_names: List[str], scale_features: bool = False) -> tuple[np.ndarray, np.ndarray]:
    model.eval()
    
    X = input_data
    if scale_features:
        X = feature_pipeline.scale_features(X, feature_names)
    
    X = tensorfy(X).to(device)
    
    with torch.no_grad():
        temp_pred, wind_pred = model(X)
        temp_pred = temp_pred.cpu().numpy()
        if scale_features:
            temp_pred = feature_pipeline.inverse_scale_features(temp_pred)
        return temp_pred, wind_pred.cpu().numpy()

In [10]:
def plot_losses(history):
    plt.figure(figsize=(10, 6))
    plt.plot(np.log(history['train_loss']), label='Training Loss')
    plt.plot(np.log(history['val_loss']), label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training History')
    plt.legend()
    plt.grid(True)
    plt.show()

In [11]:
def compare_temp(city: str, y_temp_test_np: np.ndarray, outputs_temp_test_np: np.ndarray):
    plt.figure(figsize=(10,6))
    plt.scatter(y_temp_test_np, outputs_temp_test_np, alpha=0.5, label='Test Samples')
    plt.plot([y_temp_test_np.min(), y_temp_test_np.max()], 
             [y_temp_test_np.min(), y_temp_test_np.max()], 
             'r--', label='Ideal Line')
    plt.xlabel('Actual Temperature (°C)')
    plt.ylabel('Predicted Temperature (°C)')
    plt.title(f'Actual vs Predicted Temperature for {city} - Test')
    plt.legend()
    plt.grid(True)
    plt.show()


In [12]:
def plot_temp_in_time(city: str, y_temp_test_np: np.ndarray, outputs_temp_test_np: np.ndarray):
    plt.figure(figsize=(14,6))
    plt.plot(y_temp_test_np, label='Actual Temperature', color='blue')
    plt.plot(outputs_temp_test_np, label='Predicted Temperature', color='orange')

    mae = np.mean(np.abs(y_temp_test_np - outputs_temp_test_np))

    differences = np.abs(y_temp_test_np - outputs_temp_test_np)
    mask_high_diff = differences > 2
    plt.scatter(np.where(mask_high_diff)[0], y_temp_test_np[mask_high_diff],
                color='red', label='Difference > 2°C', zorder=5)

    plt.xlabel('Test Samples')
    plt.ylabel('Temperature (°C)')
    plt.title(f'Actual vs Predicted Temperature Over Time for {city} - Test - MAE: {mae:.2f}')
    plt.legend()
    plt.grid(True)
    plt.show()

In [13]:
def run_weather_prediction(city: str, city_data: dict, feature_pipeline, scale_features: bool = False, optimizer_name: str = "sgd", n_epochs: int = 10_000):
    model, history = create_and_train_model(
        train_dataset=city_data["train"],
        val_dataset=city_data["val"],
        optimizer_name=optimizer_name,
        feature_pipeline=feature_pipeline,
        scale_features=scale_features,
        n_epochs=n_epochs
    )
    
    temp_pred, wind_pred = predict(
        model=model,
        feature_pipeline=feature_pipeline,
        input_data=city_data["test"]["X"],
        scale_features=scale_features
    )
    
    plot_losses(history)
    compare_temp(city, city_data["test"]["y_temp"], temp_pred)
    plot_temp_in_time(city, city_data["test"]["y_temp"], temp_pred)
    
    return model, history, (temp_pred, wind_pred)

In [ ]:
pipeline = WeatherPipeline()
config = DatasetConfig(
    test_size=0.15,
    val_size=0.15,
    window_size=3,
    prediction_offset=1
)

# Prepare datasets (this will also load/initialize the feature pipeline)
city_datasets = pipeline.prepare_datasets(config=config, load_windowed=True)

city = "Vancouver"
_ = run_weather_prediction(city, city_datasets[city], pipeline.feature_pipeline, scale_features=False, optimizer_name="sgd", n_epochs=50000)


Loading pre-windowed data for Albuquerque...
Loading pre-windowed data for Atlanta...
Loading pre-windowed data for Beersheba...
Loading pre-windowed data for Boston...
Loading pre-windowed data for Charlotte...
Loading pre-windowed data for Chicago...
Loading pre-windowed data for Dallas...
Loading pre-windowed data for Denver...
Loading pre-windowed data for Detroit...
Loading pre-windowed data for Eilat...
Loading pre-windowed data for Haifa...
Loading pre-windowed data for Houston...
Loading pre-windowed data for Indianapolis...
Loading pre-windowed data for Jacksonville...
Loading pre-windowed data for Jerusalem...
Loading pre-windowed data for Kansas City...
Loading pre-windowed data for Las Vegas...
Loading pre-windowed data for Los Angeles...
Loading pre-windowed data for Miami...
Loading pre-windowed data for Minneapolis...
Loading pre-windowed data for Montreal...
Loading pre-windowed data for Nahariyya...
Loading pre-windowed data for Nashville...
Loading pre-windowed data f

ValueError: X has 8 features, but StandardScaler is expecting 1 features as input.